# `pyhf` Demo

# Hello World, `pyhf` style

**Two bin counting experiment with a background uncertainty:**

In [1]:
import pyhf
import pyhf.simplemodels
import pyhf.utils

In [2]:
print('Using pyhf version {}'.format(pyhf.__version__))

Using pyhf version 0.0.15


In [3]:
pdf = pyhf.simplemodels.hepdata_like(signal_data=[12.,11.], bkg_data=[50.,52.], bkg_uncerts=[3.,7.])
CLs_obs, CLs_exp = pyhf.utils.hypotest(
        1.0, [51, 48] + pdf.config.auxdata, pdf,
        return_expected=True)
print('Observed: {} Expected: {}'.format(CLs_obs, CLs_exp))
numpy_results = CLs_obs, CLs_exp

Observed: [0.05290116] Expected: [0.06445521]


**What backend is being used?**

In [4]:
pyhf.get_backend()

(<pyhf.tensor.numpy_backend.numpy_backend at 0x7f71a42bd390>,
 <pyhf.optimize.opt_scipy.scipy_optimizer at 0x7f71a42bd470>)

**Switch out to a different backend**

In [5]:
# TensorFlow
import tensorflow as tf
sess = tf.Session()
pyhf.set_backend(pyhf.tensor.tensorflow_backend(session=sess))

In [6]:
pyhf.get_backend()

(<pyhf.tensor.tensorflow_backend.tensorflow_backend at 0x7f7172598c88>,
 <pyhf.optimize.opt_tflow.tflow_optimizer at 0x7f716394f7f0>)

**and reproduce the same result as with the NumPy backend**

In [7]:
CLs_obs, CLs_exp = pyhf.utils.hypotest(1.0, [51, 48] + pdf.config.auxdata, pdf, return_expected=True)
print('Observed: {} Expected: {}'.format(sess.run(CLs_obs), sess.run(CLs_exp)))
tensorflow_results = sess.run(CLs_obs), sess.run(CLs_exp)

/home/mcf/miniconda3/envs/pyhf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mcf/miniconda3/envs/pyhf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mcf/miniconda3/envs/pyhf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Observed: [0.05257673] Expected: [0.06445909]


**A comparison:**

In [8]:
backends = ['NumPy', 'TensorFlow']
results = [numpy_results, tensorflow_results]
for backend, result in zip(backends, results):
    print('\n# {}\nObserved: {} Expected: {}'.format(backend, result[0], result[1]))


# NumPy
Observed: [0.05290116] Expected: [0.06445521]

# TensorFlow
Observed: [0.05257673] Expected: [0.06445909]


Differences result of using single float precision versus double float precision (WIP to harmonize)

# $CL_{s}$ Example using pyhf CLI

**Use some preexisiting files**

In [9]:
# Use some shell magics in Jupyter
% ls *.json

demo.json  new_signal.json


**JSON defining a single channel, two bin counting experiment with systematics**

In [10]:
% cat demo.json

{
    "channels": [{
        "name": "singlechannel",
        "samples": [{
                "name": "sig",
                "data": [12.0, 11.0],
                "modifiers": [{
                    "name": "mu",
                    "data": null,
                    "type": "normfactor"
                }]
            },
            {
                "name": "bkg",
                "data": [50.0, 52.0],
                "modifiers": [{
                    "name": "uncorr_bkguncrt",
                    "data": [3.0, 7.0],
                    "type": "shapesys"
                }]
            }
        ]
    }],
    "data": {
        "singlechannel": [51.0, 48.0]
    },
    "toplvl": {
        "measurements": [{
            "config": {
                "poi": "mu"
            },
            "name": "singlechannel"
        }]
    }
}


In [11]:
# Use more shell magics to run from the command line
! pyhf cls demo.json

{
    "CLs_exp": [
        0.0026064088679956573,
        0.013820657528619459,
        0.06445521290832801,
        0.2352610362693783,
        0.5730418174887743
    ],
    "CLs_obs": 0.05290116224852556
}


**Can also pipe a file to `pyhf` (think [HEPData](https://hepdata.net/))**

In [12]:
json_url="https://raw.githubusercontent.com/diana-hep/pyhf/talk/DIANA-HEP-talk/docs/examples/notebooks/talks/demo.json"
! echo "{json_url}"

https://raw.githubusercontent.com/diana-hep/pyhf/talk/DIANA-HEP-talk/docs/examples/notebooks/talks/demo.json


In [13]:
! curl "{json_url}" | pyhf cls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   836  100   836    0     0   2999      0 --:--:-- --:--:-- --:--:--  2996
{
    "CLs_exp": [
        0.0026064088679956573,
        0.013820657528619459,
        0.06445521290832801,
        0.2352610362693783,
        0.5730418174887743
    ],
    "CLs_obs": 0.05290116224852556
}


# $CL_{s}$ with Reinterpretation

**Original output**

In [14]:
! pyhf cls demo.json | jq .CLs_obs

0.05290116224852556


**Consider a new signal to test**

In [15]:
% cat new_signal.json

[{
    "op": "replace",
    "path": "/channels/0/samples/0/data",
    "value": [5.0, 6.0]
}]


**Apply the patch with the new signal to update the likelihood: $L \to L'$**

- Using the [RFC 6902 (JSON Patch) standard](https://tools.ietf.org/html/rfc6902)

In [16]:
! pyhf cls demo.json --patch new_signal.json | jq .CLs_obs

0.34015787575273343
